# Data Integration
## Task 3

Date: 30 AUGUST 2021

Version: 1.0

Environment: Python 3 and Jupyter notebook

Libraries used: 
* pandas
* re
* numpy


## Introduction
All the schema conflicts were successfully resolved
Semantic mapping was performed and two datasets were merged. A global schema was declared according to the dataset obtained after completing task 1 and 2. All other incorrect datatypes or column naming, missing values, duplications, and data conflict issues were solved. Moreover, ```Id``` column was defined as the global unique key for the final dataframe. At the end, the data was stored and saved to 'dataset_integrated.csv'


##  Import libraries 

In [1]:
# Code to import libraries as you need in this assessment
import pandas as pd
import numpy as np
import re

### 1. Examining and loading data
Examine "dataset2.csv" the structure and schema. 

In [2]:
# Code to inspect the provided data file
# Couple of notes for all code block in this notebook
## please provide proper comment on your code
## Please re-start and run all cells to make sure codes are runable and include your output in the submission
data1 = pd.read_csv('dataset1_solution.csv')
data2 = pd.read_csv('dataset2.csv')

print(data1.shape)
data1.head()

(50702, 12)


,Unnamed: 0,Id,Title,Location,Company,ContractTime,ContractType,Category,Salary,OpenDate,CloseDate,SourceName
0,0,68285763,Account Manager South West Renal,UK,Orrery Recruitment Ltd,permanent,non-specified,Healthcare & Nursing Jobs,45000.0,2013-08-08 00:00:00,2013-11-06 00:00:00,emedcareers.com
1,1,70344003,SCRA / SCRAII Progression,London,RBW Consulting Solutions Ltd,permanent,non-specified,Healthcare & Nursing Jobs,38000.0,2012-04-01 12:00:00,2012-06-30 12:00:00,emedcareers.com
2,2,69266181,Critical Care Advisor (Insurance Sector),London,Chadwick Nott,permanent,non-specified,Healthcare & Nursing Jobs,22500.0,2012-08-16 15:00:00,2012-10-15 15:00:00,emedcareers.com
3,3,71713141,Scientific Director Medical Communications,Manchester,Executive Facilities Limited,permanent,non-specified,Healthcare & Nursing Jobs,50000.0,2012-05-08 12:00:00,2012-06-07 12:00:00,emedcareers.com
4,4,68100927,Territory Manager Critical Care M62 Corridor,North West England,Remtec Search and Selection,permanent,non-specified,Healthcare & Nursing Jobs,36000.0,2013-05-09 15:00:00,2013-07-08 15:00:00,emedcareers.com


In [3]:
print(data2.shape)
data2.head()

(5000, 9)


,Job Title,Organisation,Monthly Payment,Contract Type,Category,Closing,Location,Full-Time Equivalent (FTE),Opening
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,2750.00,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,FULL TIME,2012-12-24 15:00:00
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,2583.33,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,FULL TIME,2012-10-15 15:00:00
2,Deputy Manager Nursing Home (RGN) London,Liquid Personnel Ltd,3000.00,NaN,Healthcare and Nursing,2012-12-08 00:00:00,London,FULL TIME,2012-11-08 00:00:00
3,Deputy Manager (RGN) Nursing Home Bristol,Liquid Personnel Ltd,2791.67,NaN,Healthcare and Nursing,2013-01-13 15:00:00,Bristol,FULL TIME,2012-11-14 15:00:00
4,RGN or RMN Wigan,Eden Brown,2240.00,NaN,Healthcare and Nursing,2013-01-24 15:00:00,Wigan,FULL TIME,2012-11-25 15:00:00


###  Defining Initial Global Schema
Looking at the above two schemas i.e., data1 and data2, there were many conflicts found. These are listed below:
* data2 doesnot consist of `Id`, `ContractTime`, `Salary`(yearly), and `SourceName` columns and data1 doesnot consist of `Full-Time Equivalent (FTE)` and `Monthly Payment`.
* Few columns in the two schema's are same such as:
    - `Title`(data1) and `Job Title`(data2): with same datatype 
    - `Company`(data1) and `Organization`(data2): with same datatype 
    - `ContractType`(data1) and `Contract Type`(data2):  with same datatype but different entries, for example, `ContractType` consists of 'full_time', 'part_time' and 'non-specified' entries whereas `Contract Type` consists of 'Permanent' and 'Fixed Term Contract' entries
    - `OpenDate`(data1) and `Opening`(data2): different datatype but similar entries
    - `CloseDate`(data1) and `Closing`(data2): different datatype but similar entries
* If `ContractTime` and `Contract Type` columns are considered from data1 and data2 respectively, it was found that these two columns have similar entries e.g., 'permanent' and 'contract' in data1 and 'Permanent' and 'Fixed Term Contract' in data2.
* `Salary` column from data1 and `Monthly Payment` column from data2 are both annual and monthly salaries of the advertised job position.
* `Category` in both datatsets consists of same information but has different format for the written content. After looking at the entries in these columns, 
  - It was observed that `Category` from data1 consists of following entries: 
    ```Python
    IT Jobs                             
    Healthcare & Nursing Jobs            
    Engineering Jobs                     
    Accounting & Finance Jobs            
    Sales Jobs                           
    Hospitality & Catering Jobs          
    Teaching Jobs                        
    PR, Advertising & Marketing Jobs  ```  
  - The `Category` column of data2 has following entries: 
    ```Python
    Information Technology           
    Engineering                      
    Healthcare and Nursing            
    Finance and Accounting            
    Sales                             
    Hospitality and Catering          
    Teaching                          
    PR, Advertising and Marketing  ```    
* `OpenDate` and `CloseDate` columns from data1 and `Opening` and `Closing` columns from data2 represent the same datetime in 'yyyy-mm-dd hh:mm:ss' format but the datatype of data2 columns: `Opening` and `Closing` is object rather than datetime64.
* Since FTE is the number of hours worked in a week, an entry *'FULL TIME'* in the `Full-Time Equivalent (FTE)` column of data2 would mean that it is a the ad posted for a job is full time employment whereas other entires such as *'0.2 FTE'*, *'0.4 FTE'*, *'0.6 FTE'*, or *'0.8 FTE'* would mean a part-time job ab. Therefore, `Full-Time Equivalent (FTE)` column of data2 is identical to `ContractType` column of data1.
* ```Id``` and ```SourceName``` columns are missing in data2


<span style="color: purple"> <b> Now, defining a global schema: </b> </span>
* ```Id``` : *8 digit Id of the job advertisement*
*  ```Title```: *Title of the advertised job position*
*  ```Location```: *Location of the advertised job position*
*  ```Company```: *Company (employer) of the advertised job position*
*  ```ContractType```: *The contract type of the advertised job position*
*  ```ContractTime```: *The contract time of the advertised job position*
*  ```Category```: *The category of the advertised job position*
*  ```Salary```: *Annual salary of the advertised job position*
*  ```OpenDate```: *The opening time in 'yyyy-mm-dd hh:mm:ss' for the job application*
*  ```CloseDate```: *The closing time in 'yyyy-mm-dd hh:mm:ss'for applying for the advertised job position*
*  ```SourceName```: *The website where the job position is advertised* 

<span style="color: red"> Note: It is assumed that data2 is already clean </span>

In [4]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50702 entries, 0 to 50701
Data columns (total 12 columns):
Unnamed: 0      50702 non-null int64
Id              50702 non-null int64
Title           50702 non-null object
Location        50702 non-null object
Company         50702 non-null object
ContractTime    50702 non-null object
ContractType    50702 non-null object
Category        50702 non-null object
Salary          50702 non-null float64
OpenDate        50702 non-null object
CloseDate       50702 non-null object
SourceName      50702 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 4.6+ MB


In [5]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
Job Title                     5000 non-null object
Organisation                  4937 non-null object
Monthly Payment               5000 non-null float64
Contract Type                 4312 non-null object
Category                      5000 non-null object
Closing                       5000 non-null object
Location                      5000 non-null object
Full-Time Equivalent (FTE)    643 non-null object
Opening                       5000 non-null object
dtypes: float64(1), object(8)
memory usage: 351.7+ KB


In [6]:
data2.isna().sum()

Job Title                        0
Organisation                    63
Monthly Payment                  0
Contract Type                  688
Category                         0
Closing                          0
Location                         0
Full-Time Equivalent (FTE)    4357
Opening                          0
dtype: int64

In [7]:
data2['Job Title'].value_counts()

Dispensing Opticians Jobs in Falmouth                                   1
Principal / Associate (Development Infrastructure)                      1
Trainee Data Analyst                                                    1
Test Development Engineer Teradyne  Cambridge                           1
Develop into Regulatory Affairs Project Manager                         1
                                                                       ..
St Petersburg, Experienced Nanny for newborn  2 yr old girl, Sep Acc    1
Midwives  Australia                                                     1
Microsoft Server Administrator                                          1
Fund Accounting Specialist                                              1
CAD/Solidworks Draughtsman                                              1
Name: Job Title, Length: 5000, dtype: int64

In [8]:
data2['Organisation'].value_counts()

UKStaffsearch                              72
Flame Health Associates LLP                42
CHERRY RED RECRUITMENT                     39
Jobsite Jobs                               37
Hays                                       35
                                           ..
Ten Live Limited                            1
Aspire People Limited                       1
Nexers                                      1
Talent Hub Resourcing Solutions Limited     1
Thornhvac                                   1
Name: Organisation, Length: 2205, dtype: int64

In [9]:
data2['Monthly Payment'].value_counts()

3333.33    196
2916.67    185
2500.00    183
2708.33    161
3125.00    155
          ... 
1356.25      1
620.00       1
3610.25      1
1008.00      1
1658.33      1
Name: Monthly Payment, Length: 617, dtype: int64

In [10]:
data2['Contract Type'].value_counts()

Permanent              3720
Fixed Term Contract     592
Name: Contract Type, dtype: int64

In [11]:
data1.Category.value_counts()

IT Jobs                             13125
Healthcare & Nursing Jobs            8184
Engineering Jobs                     7198
Accounting & Finance Jobs            6809
Sales Jobs                           4747
Hospitality & Catering Jobs          4530
Teaching Jobs                        3557
PR, Advertising & Marketing Jobs     2552
Name: Category, dtype: int64

In [12]:
data2['Category'].value_counts()

Information Technology           1286
Engineering                      1066
Healthcare and Nursing            655
Finance and Accounting            637
Sales                             630
Hospitality and Catering          275
Teaching                          234
PR, Advertising and Marketing     217
Name: Category, dtype: int64

In [13]:
data2['Closing'].value_counts()

2013-12-25 12:00:00    8
2013-01-02 00:00:00    8
2013-06-26 00:00:00    8
2012-05-23 15:00:00    8
2013-06-02 15:00:00    8
                      ..
2012-07-04 12:00:00    1
2014-02-28 15:00:00    1
2013-02-09 15:00:00    1
2012-04-14 12:00:00    1
2014-03-18 15:00:00    1
Name: Closing, Length: 2031, dtype: int64

In [14]:
data2['Location'].value_counts()

UK                   835
London               603
South East London    288
The City             133
Surrey                82
                    ... 
Greenwich              1
Morpeth                1
Chertsey               1
Haslemere              1
Chipping Norton        1
Name: Location, Length: 395, dtype: int64

In [15]:
data2['Full-Time Equivalent (FTE)'].value_counts()

FULL TIME    583
0.2 FTE       19
0.4 FTE       16
0.8 FTE       14
0.6 FTE       11
Name: Full-Time Equivalent (FTE), dtype: int64

In [16]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50702 entries, 0 to 50701
Data columns (total 12 columns):
Unnamed: 0      50702 non-null int64
Id              50702 non-null int64
Title           50702 non-null object
Location        50702 non-null object
Company         50702 non-null object
ContractTime    50702 non-null object
ContractType    50702 non-null object
Category        50702 non-null object
Salary          50702 non-null float64
OpenDate        50702 non-null object
CloseDate       50702 non-null object
SourceName      50702 non-null object
dtypes: float64(1), int64(2), object(9)
memory usage: 4.6+ MB


### 2. Resolving schema conflicts
Based on the defined global schema, the schema conflicts are resolved one by one as shown below:

#### Conflict 1: Naming conflict in ```Title``` column in data1 vs. ```Job Title``` column in data2 & ```Company``` in data1 and   ```Organisation``` column in data2
* The naming is different in both datasets for the job ad name i.e., data1 doesnot have ```Job Title``` and data2 doesnot have ```Title```. The global schema defines ```Title``` with the same format as in data1, ```Job Title``` in data2 is replaced by ```Title```.
* The naming is different in both datasets for the company name i.e., data1 doesnot have ```Organisation``` and data2 doesnot have ```Company```. The global schema defines ```Company``` with the same format as in data1, ```Organisation``` in data2 is replaced by ```Company```.

In [17]:
# Code to inspect data and identify conflicts in schema
data2['Job Title'].value_counts()
data2['Job Title'].head()

0              Nursing Home Manager  Thames Ditton
1    Deputy Manager  Nursing Home  RGN  Shrewsbury
2       Deputy Manager  Nursing Home (RGN)  London
3      Deputy Manager (RGN)  Nursing Home  Bristol
4                                RGN or RMN  Wigan
Name: Job Title, dtype: object

The steps to resolve this conflict is by:
* Renaming ```Job Title``` by ```Title```
* Renaming ```Organisation``` by ```Company```
* Converting the datatype of ```Title```and ```Company``` to string

In [18]:
# Code to fix the conflict
data2.rename(columns = {'Job Title':'Title', 'Organisation':'Company'}, inplace = True)
data2['Title'] = data2['Title'].astype('str')
data2['Company'] = data2['Company'].astype('str')
data2.head(2)

,Title,Company,Monthly Payment,Contract Type,Category,Closing,Location,Full-Time Equivalent (FTE),Opening
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,2750.00,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,FULL TIME,2012-12-24 15:00:00
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,2583.33,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,FULL TIME,2012-10-15 15:00:00


#### Conflict 2: ```Salary``` in data1 and  ```Monthly Payment``` in data2
The ```Salary``` column in data1 consists of Yearly salary information whereas ```Monthly Payment``` contains monthly payment for the job ad. 

In [19]:
data2['Monthly Payment'].head()

0    2750.00
1    2583.33
2    3000.00
3    2791.67
4    2240.00
Name: Monthly Payment, dtype: float64

The steps to resolve this conflict is given below:
* define a function to convert the monthly payment to yearly payment
* apply this function to every row of the ```Monthly Payment``` column of data2 and store in new variable ```Salary``` to match the global schema
* delete ```Monthly Payment``` column

In [20]:
def toYearlySalary(monthlyPayment):
    return monthlyPayment * 12

data2['Salary'] = data2.apply(lambda row: toYearlySalary(row['Monthly Payment']),axis = 1)

In [21]:
data2['Salary'].head()

0    33000.00
1    30999.96
2    36000.00
3    33500.04
4    26880.00
Name: Salary, dtype: float64

In [22]:
del data2['Monthly Payment']

In [23]:
data2.head(2)

,Title,Company,Contract Type,Category,Closing,Location,Full-Time Equivalent (FTE),Opening,Salary
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,FULL TIME,2012-12-24 15:00:00,33000.00
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,FULL TIME,2012-10-15 15:00:00,30999.96


#### Conflict 3: ```ContractType``` in data1 and  ```Full-Time Equivalent (FTE)``` in data2
```Full-Time Equivalent (FTE)``` contains 'FULL TIME','0.2 FTE', '0.4 FTE', '0.8 FTE', and '0.6 FTE' entries where '0.2 FTE', '0.4 FTE', '0.8 FTE', and '0.6 FTE' represent part-time job ads. Therefore, ```Full-Time Equivalent (FTE)``` is similar to ```ContractType``` in data1.

In [24]:
data2['Full-Time Equivalent (FTE)'].value_counts()

FULL TIME    583
0.2 FTE       19
0.4 FTE       16
0.8 FTE       14
0.6 FTE       11
Name: Full-Time Equivalent (FTE), dtype: int64

The steps to resolve this conflicts is given below:
* Replace '0.2 FTE', '0.4 FTE', '0.8 FTE', and '0.6 FTE' by 'part_time' and 'FULL TIME' by 'full_time' to match the entries in data1 ```ContractType``` column
* Rename ```Full-Time Equivalent (FTE)``` to ```ContractType``` 

In [25]:
replace_dict = {'FULL TIME':'full_time', 
                '0.2 FTE':'part_time', 
                '0.4 FTE':'part_time', 
                '0.8 FTE':'part_time', 
                '0.6 FTE':'part_time'
                }
data2['Full-Time Equivalent (FTE)'].replace(replace_dict,inplace=True)
data2.rename(columns = {'Full-Time Equivalent (FTE)':'ContractType'}, inplace = True)
print(data2['ContractType'].value_counts())

full_time    583
part_time     60
Name: ContractType, dtype: int64


#### Conflict 4: ```ContractTime``` in data1 and  ```Contract Type``` in data2
```ContractTime``` in data1 and  ```Contract Type``` in data2 with similar information but different column names. For example, ```ContractTime``` in data1 consists of 'permanent', 'non-specified', and 'contract' whereas 
```Contract Type``` in data2 contains 'Permanent' and 'Fixed Term Contract'.

In [26]:
data2['Contract Type'].value_counts()

Permanent              3720
Fixed Term Contract     592
Name: Contract Type, dtype: int64

In [27]:
data1['ContractTime'].value_counts()

permanent        30335
non-specified    14796
contract          5571
Name: ContractTime, dtype: int64

To resolve this conflict:
* Replace 'Permanent' by 'permanent' and 'Fixed Term Contract' by 'contract'
* Rename ```Contract Type``` by ```ContractTime```

In [28]:
replace_dict = {'Permanent':'permanent', 
                'Fixed Term Contract':'contract'
               }
data2['Contract Type'].replace(replace_dict,inplace=True)
data2.rename(columns = {'Contract Type':'ContractTime'}, inplace = True)
print(data2['ContractTime'].value_counts())

permanent    3720
contract      592
Name: ContractTime, dtype: int64


In [29]:
data2.head(2)

,Title,Company,ContractTime,Category,Closing,Location,ContractType,Opening,Salary
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,full_time,2012-12-24 15:00:00,33000.00
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,full_time,2012-10-15 15:00:00,30999.96


#### Conflict 5: ```OpenDate``` and ```CloseDate``` in data1 &  ```Opening``` and ```Closing```in data2
 ```OpenDate``` and ```CloseDate``` in data1 consists of date and time of the opening and closing for applying for a particular job in a specific time period. Also, ```Opening``` and ```Closing```in data2 consist of same information but the observed datatype for these columns was object and not datetime.

In [30]:
print(data2['Opening'].dtype)
print(data2['Closing'].dtype)

object
object


To resolve this conflict:
* Rename 'Opening' to 'OpenDate' and 'Closing' to 'CloseDate'
* Change the datatype to datetime

In [31]:
data2.rename(columns = {'Opening':'OpenDate', 'Closing':'CloseDate'}, inplace = True)

In [32]:
# data2['OpenDate'] = data2['OpenDate'].apply(lambda x: x.strftime('%Y-%m-%d %H:%M:%S'))
data2['CloseDate'] = data2['CloseDate'].astype('datetime64')
data2['OpenDate'] = pd.to_datetime(data2['OpenDate'])

In [33]:
data2.head(2)

,Title,Company,ContractTime,Category,CloseDate,Location,ContractType,OpenDate,Salary
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,full_time,2012-12-24 15:00:00,33000.00
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,full_time,2012-10-15 15:00:00,30999.96


#### Conflict 5: Missing ```SourceName``` column in data2
```SourceName``` was missing in data2. All data in data2 is coming from datasource www.jobhuntlisting.com. Therefore, each jobad in this dataset was posted on www.jobhuntlisting.com.

To resolve this conflict, add a new column named "SourceName" as defined in the global schema. Each row of this column will contain *'www.jobhuntlisting.com'* as an entry.

In [34]:
data2['SourceName'] = "www.jobhuntlisting.com"

In [35]:
data2.head()

,Title,Company,ContractTime,Category,CloseDate,Location,ContractType,OpenDate,Salary,SourceName
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,full_time,2012-12-24 15:00:00,33000.00,www.jobhuntlisting.com
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,full_time,2012-10-15 15:00:00,30999.96,www.jobhuntlisting.com
2,Deputy Manager Nursing Home (RGN) London,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2012-12-08 00:00:00,London,full_time,2012-11-08 00:00:00,36000.00,www.jobhuntlisting.com
3,Deputy Manager (RGN) Nursing Home Bristol,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2013-01-13 15:00:00,Bristol,full_time,2012-11-14 15:00:00,33500.04,www.jobhuntlisting.com
4,RGN or RMN Wigan,Eden Brown,NaN,Healthcare and Nursing,2013-01-24 15:00:00,Wigan,full_time,2012-11-25 15:00:00,26880.00,www.jobhuntlisting.com


In [36]:
print("Shape of dataset1_solution.csv: ", data1.shape)

Shape of dataset1_solution.csv:  (50702, 12)


In [37]:
print("Shape of dataset2.csv: ",data2.shape)

Shape of dataset2.csv:  (5000, 10)


In [38]:
data1.head()

,Unnamed: 0,Id,Title,Location,Company,ContractTime,ContractType,Category,Salary,OpenDate,CloseDate,SourceName
0,0,68285763,Account Manager South West Renal,UK,Orrery Recruitment Ltd,permanent,non-specified,Healthcare & Nursing Jobs,45000.0,2013-08-08 00:00:00,2013-11-06 00:00:00,emedcareers.com
1,1,70344003,SCRA / SCRAII Progression,London,RBW Consulting Solutions Ltd,permanent,non-specified,Healthcare & Nursing Jobs,38000.0,2012-04-01 12:00:00,2012-06-30 12:00:00,emedcareers.com
2,2,69266181,Critical Care Advisor (Insurance Sector),London,Chadwick Nott,permanent,non-specified,Healthcare & Nursing Jobs,22500.0,2012-08-16 15:00:00,2012-10-15 15:00:00,emedcareers.com
3,3,71713141,Scientific Director Medical Communications,Manchester,Executive Facilities Limited,permanent,non-specified,Healthcare & Nursing Jobs,50000.0,2012-05-08 12:00:00,2012-06-07 12:00:00,emedcareers.com
4,4,68100927,Territory Manager Critical Care M62 Corridor,North West England,Remtec Search and Selection,permanent,non-specified,Healthcare & Nursing Jobs,36000.0,2013-05-09 15:00:00,2013-07-08 15:00:00,emedcareers.com


In [39]:
data2.head()

,Title,Company,ContractTime,Category,CloseDate,Location,ContractType,OpenDate,Salary,SourceName
0,Nursing Home Manager Thames Ditton,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2013-02-22 15:00:00,Thames Ditton,full_time,2012-12-24 15:00:00,33000.00,www.jobhuntlisting.com
1,Deputy Manager Nursing Home RGN Shrewsbury,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2012-11-14 15:00:00,Shrewsbury,full_time,2012-10-15 15:00:00,30999.96,www.jobhuntlisting.com
2,Deputy Manager Nursing Home (RGN) London,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2012-12-08 00:00:00,London,full_time,2012-11-08 00:00:00,36000.00,www.jobhuntlisting.com
3,Deputy Manager (RGN) Nursing Home Bristol,Liquid Personnel Ltd,NaN,Healthcare and Nursing,2013-01-13 15:00:00,Bristol,full_time,2012-11-14 15:00:00,33500.04,www.jobhuntlisting.com
4,RGN or RMN Wigan,Eden Brown,NaN,Healthcare and Nursing,2013-01-24 15:00:00,Wigan,full_time,2012-11-25 15:00:00,26880.00,www.jobhuntlisting.com


Before merging the above two datasets, the following steps needs to be performed:

1. Delete the ```Unnamed: 0``` column from data1.
2. Perform an inner join while merging the datasets since data2 doesnot have Id and SourceName columns therefore it would be difficult to identify any duplicated rows generated after simply concatenating. 
3. Check if there are duplicate values generated after the inner join
4. Remove those repeated rows after merging the datasets

In [40]:
del data1['Unnamed: 0']

In [41]:
inner_joined_dataset = pd.concat([data1, data2], join="inner")
inner_joined_dataset

,Title,Location,Company,ContractTime,ContractType,Category,Salary,OpenDate,CloseDate,SourceName
0,Account Manager South West Renal,UK,Orrery Recruitment Ltd,permanent,non-specified,Healthcare & Nursing Jobs,45000.00,2013-08-08 00:00:00,2013-11-06 00:00:00,emedcareers.com
1,SCRA / SCRAII Progression,London,RBW Consulting Solutions Ltd,permanent,non-specified,Healthcare & Nursing Jobs,38000.00,2012-04-01 12:00:00,2012-06-30 12:00:00,emedcareers.com
2,Critical Care Advisor (Insurance Sector),London,Chadwick Nott,permanent,non-specified,Healthcare & Nursing Jobs,22500.00,2012-08-16 15:00:00,2012-10-15 15:00:00,emedcareers.com
3,Scientific Director Medical Communications,Manchester,Executive Facilities Limited,permanent,non-specified,Healthcare & Nursing Jobs,50000.00,2012-05-08 12:00:00,2012-06-07 12:00:00,emedcareers.com
4,Territory Manager Critical Care M62 Corridor,North West England,Remtec Search and Selection,permanent,non-specified,Healthcare & Nursing Jobs,36000.00,2013-05-09 15:00:00,2013-07-08 15:00:00,emedcareers.com
...,...,...,...,...,...,...,...,...,...,...
4995,Pharmaceutical Analytical Chemist Jobs in Nor...,Northern Ireland,Flame Pharma,permanent,NaN,Healthcare and Nursing,23000.04,2013-09-24 15:00:00,2013-10-24 15:00:00,www.jobhuntlisting.com
4996,Medical Sales Representative South East Essex,UK,The Vacancy Management Company,contract,NaN,Healthcare and Nursing,22500.00,2012-05-21 12:00:00,2012-06-20 12:00:00,www.jobhuntlisting.com
4997,SENIOR PROJECT MANAGER EVENT & EXHIBITION AGENCY,Hertfordshire,Live Recruitment,permanent,full_time,"PR, Advertising and Marketing",32499.96,2013-11-17 15:00:00,2013-12-17 15:00:00,www.jobhuntlisting.com
4998,Curriculum Leader Mathematics,Wrexham,nan,permanent,NaN,Teaching,5006.04,2012-01-15 12:00:00,2012-03-15 12:00:00,www.jobhuntlisting.com


In [42]:
duplicates = inner_joined_dataset[inner_joined_dataset.duplicated(keep=False)] # showing all duplicated records
print ("There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 4 duplicate records found


,Title,Location,Company,ContractTime,ContractType,Category,Salary,OpenDate,CloseDate,SourceName
40185,Client Service Accountant (Fully Qualified),Edinburgh,Clearwater Analytics,permanent,non-specified,Accounting & Finance Jobs,39500.0,2013-07-07 00:00:00,2013-10-05 00:00:00,tntjobs.co.uk
40253,Client Service Accountant (Fully Qualified),Edinburgh,Clearwater Analytics,permanent,non-specified,Accounting & Finance Jobs,39500.0,2013-07-07 00:00:00,2013-10-05 00:00:00,tntjobs.co.uk
28696,Senior Credit Risk Modeller Leeds,West Yorkshire,Aspire Data Recruitment,permanent,full_time,IT Jobs,41750.0,2013-09-18 15:00:00,2013-12-17 15:00:00,planetrecruit.com
28877,Senior Credit Risk Modeller Leeds,West Yorkshire,Aspire Data Recruitment,permanent,full_time,IT Jobs,41750.0,2013-09-18 15:00:00,2013-12-17 15:00:00,planetrecruit.com


Keeping the above four rows in mind, these rows can be removed after normally merging the two datasets. 

### 3. Merging data



In [43]:
# Code to merge two data sets
df = pd.concat([data2, data1], ignore_index=True)
df

/Users/pallavibhimte/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,Category,CloseDate,Company,ContractTime,ContractType,Id,Location,OpenDate,Salary,SourceName,Title
0,Healthcare and Nursing,2013-02-22 15:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,Thames Ditton,2012-12-24 15:00:00,33000.00,www.jobhuntlisting.com,Nursing Home Manager Thames Ditton
1,Healthcare and Nursing,2012-11-14 15:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,Shrewsbury,2012-10-15 15:00:00,30999.96,www.jobhuntlisting.com,Deputy Manager Nursing Home RGN Shrewsbury
2,Healthcare and Nursing,2012-12-08 00:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,London,2012-11-08 00:00:00,36000.00,www.jobhuntlisting.com,Deputy Manager Nursing Home (RGN) London
3,Healthcare and Nursing,2013-01-13 15:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,Bristol,2012-11-14 15:00:00,33500.04,www.jobhuntlisting.com,Deputy Manager (RGN) Nursing Home Bristol
4,Healthcare and Nursing,2013-01-24 15:00:00,Eden Brown,NaN,full_time,NaN,Wigan,2012-11-25 15:00:00,26880.00,www.jobhuntlisting.com,RGN or RMN Wigan
...,...,...,...,...,...,...,...,...,...,...,...
55697,Healthcare & Nursing Jobs,2012-06-13 15:00:00,non-specified,permanent,non-specified,69005978.0,Scotland,2012-04-14 15:00:00,55752.00,scotsman.com,Relations Officer
55698,Sales Jobs,2013-12-28 12:00:00,non-specified,permanent,non-specified,70769728.0,UK,2013-10-29 12:00:00,11000.00,scotsman.com,Magazine Advertising Sales Executive (Sports &...
55699,IT Jobs,2012-03-12 12:00:00,UNKNOWN auto added by vacany import,non-specified,non-specified,68704692.0,London,2012-01-12 12:00:00,25000.00,grb.uk.com,Graduate SoftwareWeb Developer
55700,Healthcare & Nursing Jobs,2012-09-05 12:00:00,University Hospitals Southampton NHS Foundatio...,permanent,non-specified,72342224.0,Southampton,2012-06-07 12:00:00,31227.00,hsjjobs.com,Care Group Manager – Emergency Care


### 4. Resolving data conflicts:


#### Conflict 1: Data Duplication


In [44]:
df.shape

(55702, 11)

In [45]:
# Code to fix the conflict
# Looking for duplicate values in 'Title', 'Location', 'Company', 'ContractTime', 'ContractType', 'Category', 'Salary', 'OpenDate', 'CloseDate' columns
df.drop_duplicates(subset=['Title', 'Location', 'Company', 
                           'ContractTime', 'ContractType', 'Category', 
                           'Salary', 'OpenDate', 'CloseDate'], 
                   inplace=True)
df

,Category,CloseDate,Company,ContractTime,ContractType,Id,Location,OpenDate,Salary,SourceName,Title
0,Healthcare and Nursing,2013-02-22 15:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,Thames Ditton,2012-12-24 15:00:00,33000.00,www.jobhuntlisting.com,Nursing Home Manager Thames Ditton
1,Healthcare and Nursing,2012-11-14 15:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,Shrewsbury,2012-10-15 15:00:00,30999.96,www.jobhuntlisting.com,Deputy Manager Nursing Home RGN Shrewsbury
2,Healthcare and Nursing,2012-12-08 00:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,London,2012-11-08 00:00:00,36000.00,www.jobhuntlisting.com,Deputy Manager Nursing Home (RGN) London
3,Healthcare and Nursing,2013-01-13 15:00:00,Liquid Personnel Ltd,NaN,full_time,NaN,Bristol,2012-11-14 15:00:00,33500.04,www.jobhuntlisting.com,Deputy Manager (RGN) Nursing Home Bristol
4,Healthcare and Nursing,2013-01-24 15:00:00,Eden Brown,NaN,full_time,NaN,Wigan,2012-11-25 15:00:00,26880.00,www.jobhuntlisting.com,RGN or RMN Wigan
...,...,...,...,...,...,...,...,...,...,...,...
55697,Healthcare & Nursing Jobs,2012-06-13 15:00:00,non-specified,permanent,non-specified,69005978.0,Scotland,2012-04-14 15:00:00,55752.00,scotsman.com,Relations Officer
55698,Sales Jobs,2013-12-28 12:00:00,non-specified,permanent,non-specified,70769728.0,UK,2013-10-29 12:00:00,11000.00,scotsman.com,Magazine Advertising Sales Executive (Sports &...
55699,IT Jobs,2012-03-12 12:00:00,UNKNOWN auto added by vacany import,non-specified,non-specified,68704692.0,London,2012-01-12 12:00:00,25000.00,grb.uk.com,Graduate SoftwareWeb Developer
55700,Healthcare & Nursing Jobs,2012-09-05 12:00:00,University Hospitals Southampton NHS Foundatio...,permanent,non-specified,72342224.0,Southampton,2012-06-07 12:00:00,31227.00,hsjjobs.com,Care Group Manager – Emergency Care


From the above output, it is evident that two duplicate rows were deleted from the dataset as the dimensions of the datset previously was (55702 x 11) and after dropping the duplicates, it is (55700 x 11). 

In [46]:
data1.Id.value_counts()

69476348    1
55410042    1
68696694    1
69680498    1
69545328    1
           ..
68994994    1
68825007    1
69216174    1
68695980    1
68824827    1
Name: Id, Length: 50702, dtype: int64

In [47]:
df['Id'].min()

12612628.0

In [48]:
df['Id'].max()

72695947.0

In [49]:
list1 = np.random.choice(range(98765432), 5000, replace=False)

In [50]:
list1

array([25845065,  4949411, 20140870, ..., 53857757, 48331600, 47359502])

In [51]:
df.loc[df.Id.isnull(), 'Id'] = list1

In [52]:
df

,Category,CloseDate,Company,ContractTime,ContractType,Id,Location,OpenDate,Salary,SourceName,Title
0,Healthcare and Nursing,2013-02-22 15:00:00,Liquid Personnel Ltd,NaN,full_time,25845065.0,Thames Ditton,2012-12-24 15:00:00,33000.00,www.jobhuntlisting.com,Nursing Home Manager Thames Ditton
1,Healthcare and Nursing,2012-11-14 15:00:00,Liquid Personnel Ltd,NaN,full_time,4949411.0,Shrewsbury,2012-10-15 15:00:00,30999.96,www.jobhuntlisting.com,Deputy Manager Nursing Home RGN Shrewsbury
2,Healthcare and Nursing,2012-12-08 00:00:00,Liquid Personnel Ltd,NaN,full_time,20140870.0,London,2012-11-08 00:00:00,36000.00,www.jobhuntlisting.com,Deputy Manager Nursing Home (RGN) London
3,Healthcare and Nursing,2013-01-13 15:00:00,Liquid Personnel Ltd,NaN,full_time,80451482.0,Bristol,2012-11-14 15:00:00,33500.04,www.jobhuntlisting.com,Deputy Manager (RGN) Nursing Home Bristol
4,Healthcare and Nursing,2013-01-24 15:00:00,Eden Brown,NaN,full_time,4469749.0,Wigan,2012-11-25 15:00:00,26880.00,www.jobhuntlisting.com,RGN or RMN Wigan
...,...,...,...,...,...,...,...,...,...,...,...
55697,Healthcare & Nursing Jobs,2012-06-13 15:00:00,non-specified,permanent,non-specified,69005978.0,Scotland,2012-04-14 15:00:00,55752.00,scotsman.com,Relations Officer
55698,Sales Jobs,2013-12-28 12:00:00,non-specified,permanent,non-specified,70769728.0,UK,2013-10-29 12:00:00,11000.00,scotsman.com,Magazine Advertising Sales Executive (Sports &...
55699,IT Jobs,2012-03-12 12:00:00,UNKNOWN auto added by vacany import,non-specified,non-specified,68704692.0,London,2012-01-12 12:00:00,25000.00,grb.uk.com,Graduate SoftwareWeb Developer
55700,Healthcare & Nursing Jobs,2012-09-05 12:00:00,University Hospitals Southampton NHS Foundatio...,permanent,non-specified,72342224.0,Southampton,2012-06-07 12:00:00,31227.00,hsjjobs.com,Care Group Manager – Emergency Care


#### Finding global key for the data
Find a proper global key for the integrated data and give justification here.

In [53]:
# Code to identify global key
df.Id.value_counts()

67106278.0    2
69688504.0    2
66700453.0    2
68028227.0    1
71794719.0    1
             ..
69546609.0    1
71439810.0    1
69247912.0    1
69959615.0    1
68347375.0    1
Name: Id, Length: 55697, dtype: int64

#### Conflict 2: Other duplication
As numpy generates random numbers for list1, there might be a possibility of getting duplicate number in the ```Id``` column. Therefore, to be sure that there is no repeated value, the below code is used to drop any duplicates from the ```Id``` column.

In [54]:
df.duplicated(subset=['Id']).sum()

3

In [55]:
df.drop_duplicates(['Id'])

,Category,CloseDate,Company,ContractTime,ContractType,Id,Location,OpenDate,Salary,SourceName,Title
0,Healthcare and Nursing,2013-02-22 15:00:00,Liquid Personnel Ltd,NaN,full_time,25845065.0,Thames Ditton,2012-12-24 15:00:00,33000.00,www.jobhuntlisting.com,Nursing Home Manager Thames Ditton
1,Healthcare and Nursing,2012-11-14 15:00:00,Liquid Personnel Ltd,NaN,full_time,4949411.0,Shrewsbury,2012-10-15 15:00:00,30999.96,www.jobhuntlisting.com,Deputy Manager Nursing Home RGN Shrewsbury
2,Healthcare and Nursing,2012-12-08 00:00:00,Liquid Personnel Ltd,NaN,full_time,20140870.0,London,2012-11-08 00:00:00,36000.00,www.jobhuntlisting.com,Deputy Manager Nursing Home (RGN) London
3,Healthcare and Nursing,2013-01-13 15:00:00,Liquid Personnel Ltd,NaN,full_time,80451482.0,Bristol,2012-11-14 15:00:00,33500.04,www.jobhuntlisting.com,Deputy Manager (RGN) Nursing Home Bristol
4,Healthcare and Nursing,2013-01-24 15:00:00,Eden Brown,NaN,full_time,4469749.0,Wigan,2012-11-25 15:00:00,26880.00,www.jobhuntlisting.com,RGN or RMN Wigan
...,...,...,...,...,...,...,...,...,...,...,...
55697,Healthcare & Nursing Jobs,2012-06-13 15:00:00,non-specified,permanent,non-specified,69005978.0,Scotland,2012-04-14 15:00:00,55752.00,scotsman.com,Relations Officer
55698,Sales Jobs,2013-12-28 12:00:00,non-specified,permanent,non-specified,70769728.0,UK,2013-10-29 12:00:00,11000.00,scotsman.com,Magazine Advertising Sales Executive (Sports &...
55699,IT Jobs,2012-03-12 12:00:00,UNKNOWN auto added by vacany import,non-specified,non-specified,68704692.0,London,2012-01-12 12:00:00,25000.00,grb.uk.com,Graduate SoftwareWeb Developer
55700,Healthcare & Nursing Jobs,2012-09-05 12:00:00,University Hospitals Southampton NHS Foundatio...,permanent,non-specified,72342224.0,Southampton,2012-06-07 12:00:00,31227.00,hsjjobs.com,Care Group Manager – Emergency Care


The above dataset uses ```Id``` as a global key as each combination of ```Id``` defines a unique identity

#### Conflict 3: Missing Values
There were 688 missing values in ```ContractTime``` and 4357 missing values in ```ContractType```. These missing values were already present in data2 but were not removed.
To resolve the conflict, replace all na values by non-specified.

In [56]:
df.isna().sum()

Category           0
CloseDate          0
Company            0
ContractTime     688
ContractType    4357
Id                 0
Location           0
OpenDate           0
Salary             0
SourceName         0
Title              0
dtype: int64

In [57]:
df["ContractTime"].fillna("non-specified", inplace = True)
df["ContractType"].fillna("non-specified", inplace = True)

In [58]:
df.isna().sum()

Category        0
CloseDate       0
Company         0
ContractTime    0
ContractType    0
Id              0
Location        0
OpenDate        0
Salary          0
SourceName      0
Title           0
dtype: int64

In [59]:
df.head(2)

,Category,CloseDate,Company,ContractTime,ContractType,Id,Location,OpenDate,Salary,SourceName,Title
0,Healthcare and Nursing,2013-02-22 15:00:00,Liquid Personnel Ltd,non-specified,full_time,25845065.0,Thames Ditton,2012-12-24 15:00:00,33000.00,www.jobhuntlisting.com,Nursing Home Manager Thames Ditton
1,Healthcare and Nursing,2012-11-14 15:00:00,Liquid Personnel Ltd,non-specified,full_time,4949411.0,Shrewsbury,2012-10-15 15:00:00,30999.96,www.jobhuntlisting.com,Deputy Manager Nursing Home RGN Shrewsbury


#### Conflict 4: Incorrect datatype for  ```Id``` column
The datatype for  ```Id``` is float. 
To resolve this conflict, change the datatype to integer.

In [60]:
df['Id'] = df['Id'].astype('int64')

#### Conflict 5: Round  ```Salary``` column upto two decimal places
The ```Salary``` column is rounded off upto two decimal points.

In [61]:
df['Salary'].round(2)

0        33000.00
1        30999.96
2        36000.00
3        33500.04
4        26880.00
           ...   
55697    55752.00
55698    11000.00
55699    25000.00
55700    31227.00
55701    20000.00
Name: Salary, Length: 55700, dtype: float64

In [62]:
df = df[['Id', 'Title', 'Location', 'Company', 'ContractType', 'ContractTime', 'Category', 'Salary', 'OpenDate', 'CloseDate', 'SourceName']]
df

,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,SourceName
0,25845065,Nursing Home Manager Thames Ditton,Thames Ditton,Liquid Personnel Ltd,full_time,non-specified,Healthcare and Nursing,33000.00,2012-12-24 15:00:00,2013-02-22 15:00:00,www.jobhuntlisting.com
1,4949411,Deputy Manager Nursing Home RGN Shrewsbury,Shrewsbury,Liquid Personnel Ltd,full_time,non-specified,Healthcare and Nursing,30999.96,2012-10-15 15:00:00,2012-11-14 15:00:00,www.jobhuntlisting.com
2,20140870,Deputy Manager Nursing Home (RGN) London,London,Liquid Personnel Ltd,full_time,non-specified,Healthcare and Nursing,36000.00,2012-11-08 00:00:00,2012-12-08 00:00:00,www.jobhuntlisting.com
3,80451482,Deputy Manager (RGN) Nursing Home Bristol,Bristol,Liquid Personnel Ltd,full_time,non-specified,Healthcare and Nursing,33500.04,2012-11-14 15:00:00,2013-01-13 15:00:00,www.jobhuntlisting.com
4,4469749,RGN or RMN Wigan,Wigan,Eden Brown,full_time,non-specified,Healthcare and Nursing,26880.00,2012-11-25 15:00:00,2013-01-24 15:00:00,www.jobhuntlisting.com
...,...,...,...,...,...,...,...,...,...,...,...
55697,69005978,Relations Officer,Scotland,non-specified,non-specified,permanent,Healthcare & Nursing Jobs,55752.00,2012-04-14 15:00:00,2012-06-13 15:00:00,scotsman.com
55698,70769728,Magazine Advertising Sales Executive (Sports &...,UK,non-specified,non-specified,permanent,Sales Jobs,11000.00,2013-10-29 12:00:00,2013-12-28 12:00:00,scotsman.com
55699,68704692,Graduate SoftwareWeb Developer,London,UNKNOWN auto added by vacany import,non-specified,non-specified,IT Jobs,25000.00,2012-01-12 12:00:00,2012-03-12 12:00:00,grb.uk.com
55700,72342224,Care Group Manager – Emergency Care,Southampton,University Hospitals Southampton NHS Foundatio...,non-specified,permanent,Healthcare & Nursing Jobs,31227.00,2012-06-07 12:00:00,2012-09-05 12:00:00,hsjjobs.com


### 5. Saving the integrated and reshaped data
The last part of the integration process is to export the output data to csv format, named as:
- 'dataset_integrated.csv'

In [64]:
# code to save output data
df.to_csv('dataset_integrated.csv')

## Summary 
In this task, 
* All the schema conflicts were successfully resolved
* Semantic mapping was performed 
* Two datasets were merged 
* A global schema was declared according to the dataset obtained after completing task 1 and 2. 
* All other incorrect datatypes or column naming, missing values, duplications, and data conflict issues were solved
* Moreover, ```Id``` column was defined as the global unique key for the final dataframe
* At the end, the data was stored and saved to 'dataset_integrated.csv'